### 加载模型推理预测结果

#### 加载预训练模型

In [ ]:
from tools import inference_img
from dataset_tools import ADE20KDataset
from dataset_tools import TensorDataset
from mmseg.apis import init_segmentor
from torchvision.transforms import Compose
import torchvision.transforms as transforms
from tqdm import tqdm
import torch
from mmseg.apis import init_segmentor, inference_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette



pspnet_config_file = '../configs/pspnet/pspnet_r50-d8_512x512_80k_ade20k.py'
pspnet_checkpoint_file = '../checkpoints/pspnet_r50-d8_512x512_80k_ade20k_20200615_014128-15a8b914.pth'
pspnet_model = init_segmentor(pspnet_config_file, pspnet_checkpoint_file, device='cuda:0')
    

fcn_config_file = '../configs/fcn/fcn_r50-d8_512x512_80k_ade20k.py'
fcn_checkpoint_file = '../checkpoints/fcn_r50-d8_512x512_80k_ade20k_20200614_144016-f8ac5082.pth'
fcn_model = init_segmentor(fcn_config_file, fcn_checkpoint_file, device='cuda:0')


deeplabv3plus_config_file = '../configs/deeplabv3plus/deeplabv3plus_r50-d8_512x512_80k_ade20k.py'
deeplabv3plus_checkpoint_file = '../checkpoints/deeplabv3plus_r50-d8_512x512_80k_ade20k_20200614_185028-bf1400d8.pth'
deeplabv3plus_model = init_segmentor(deeplabv3plus_config_file, deeplabv3plus_checkpoint_file, device='cuda:0')


device = 'cuda:0'

test_tensor_path = '/root/Desktop/我的网盘/inference_tensor_test/'
test_img_path = '/root/Desktop/我的网盘/test_set/img_test/'
test_label_path = '/root/Desktop/我的网盘/test_set/label/'

test_tensor_dataset = TensorDataset(test_tensor_path, test_label_path, device)
test_img_dataset = ADE20KDataset(test_img_path, test_label_path)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

original_img = test_img_dataset[22][0]
plt.show(original_img)

In [ ]:
import numpy as np

img_path = test_img_dataset.get_img_path(22)
label = test_img_dataset.get_label_as_PIL(22)
label = np.array(label, dtype = np.uint8)

pspnet_result = inference_segmentor(pspnet_model, img_path)
fcn_result = inference_segmentor(fcn_model, img_path)
deeplabv3p_result = inference_segmentor(deeplabv3plus_model, img_path)

#### 加载EnSeg模型，推理fusion结果

In [ ]:
model = torch.load('/root/Desktop/我的网盘/fusion_model_100.pth')   
model = model.to('cuda:0')

tensor_dict, annotation_tensor = test_tensor_dataset[22]
annotation_tensor -= 1 # 忽略背景类，将label范围从1-150变为0-149
    
deeplabv3p_logits_res = tensor_dict['deeplabv3p'].to(device)
pspnet_logits_res = tensor_dict['pspnet'].to(device)
fcn_logits_res = tensor_dict['fcn'].to(device)

res = model.forward(deeplabv3p_logits_res, pspnet_logits_res, fcn_logits_res) # (150, H, W)
enseg_result = res.argmax(0)